In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done


In [4]:
# Import findspark and initialize.
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),header=True, inferSchema=True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView("home_sales")


In [14]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
home4price = spark.sql("SELECT ROUND(AVG(price),2) AS AVG_PRICE_4BED,YEAR(date) AS YEAR from home_sales where bedrooms==4 GROUP BY YEAR order by YEAR desc")
home4price.show()


+--------------+----+
|AVG_PRICE_4BED|YEAR|
+--------------+----+
|     296363.88|2022|
|     301819.44|2021|
|     298353.78|2020|
|      300263.7|2019|
+--------------+----+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
home3price = spark.sql("SELECT ROUND (AVG(price),2) AS AVG_PRICE_3BED,date_built from home_sales where bedrooms==3 and bathrooms ==3 GROUP BY date_built order by date_built desc")
home3price.show()


+--------------+----------+
|AVG_PRICE_3BED|date_built|
+--------------+----------+
|     292676.79|      2017|
|     290555.07|      2016|
|      288770.3|      2015|
|     290852.27|      2014|
|     295962.27|      2013|
|     293683.19|      2012|
|     291117.47|      2011|
|     292859.62|      2010|
+--------------+----------+



In [16]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
home332price = spark.sql("SELECT ROUND (AVG(price),2) AS AVG_PRICE_3BED, date_built from home_sales where bedrooms==3 and bathrooms==3 and floors==2 and sqft_living >= 2000 GROUP BY date_built order by date_built desc")
home332price.show()


+--------------+----------+
|AVG_PRICE_3BED|date_built|
+--------------+----------+
|     280317.58|      2017|
|      293965.1|      2016|
|     297609.97|      2015|
|     298264.72|      2014|
|     303676.79|      2013|
|     307539.97|      2012|
|     276553.81|      2011|
|     285010.22|      2010|
+--------------+----------+



In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
homeviewprice = spark.sql("SELECT ROUND (AVG(price),2) AS AVG_PRICE_VIEW, view from home_sales GROUP BY view having AVG_PRICE_VIEW>=350000")

print("--- %s seconds ---" % (time.time() - start_time))
homeviewprice.show()

--- 0.07257437705993652 seconds ---
+--------------+----+
|AVG_PRICE_VIEW|view|
+--------------+----+
|    1056336.74|  85|
|     736679.93|  65|
|      755214.8|  53|
|    1080649.37|  78|
|    1053472.79|  81|
|    1058802.78|  76|
|    1137372.73|  91|
|    1026006.06|  93|
|     733780.26|  52|
|    1070444.25|  86|
|     1033536.2|  94|
|      734340.5|  57|
|     798684.82|  54|
|    1017815.92|  96|
|     970402.55|  92|
|     767036.67|  64|
|     746877.59|  61|
|    1031719.35|  88|
|     780914.67|  72|
|      791453.0|  59|
+--------------+----+
only showing top 20 rows



In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
homeviewprice = spark.sql("SELECT ROUND (AVG(price),2) AS AVG_PRICE_VIEW, view from home_sales GROUP BY view having AVG_PRICE_VIEW>=350000")

print("--- %s seconds ---" % (time.time() - start_time))
homeviewprice.show()

print("cached data ran much faster than the uncached data by 60%")

--- 0.03824877738952637 seconds ---
+--------------+----+
|AVG_PRICE_VIEW|view|
+--------------+----+
|    1056336.74|  85|
|     736679.93|  65|
|      755214.8|  53|
|    1080649.37|  78|
|    1053472.79|  81|
|    1058802.78|  76|
|    1137372.73|  91|
|    1026006.06|  93|
|     733780.26|  52|
|    1070444.25|  86|
|     1033536.2|  94|
|      734340.5|  57|
|     798684.82|  54|
|    1017815.92|  96|
|     970402.55|  92|
|     767036.67|  64|
|     746877.59|  61|
|    1031719.35|  88|
|     780914.67|  72|
|      791453.0|  59|
+--------------+----+
only showing top 20 rows

cached data ran much faster than the uncached data by 60%


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy('date_built').parquet('home_parquet', mode = 'overwrite')

In [27]:
# 11. Read the formatted parquet data.
home_parquet_df = spark.read.parquet('home_parquet')
home_parquet_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [28]:
# 12. Create a temporary table for the parquet data.
home_parquet_df.createOrReplaceTempView('tempparquet')


In [29]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
homeparq = spark.sql("SELECT ROUND (AVG(price),2) AS AVG_PRICE_VIEW, view from home_sales GROUP BY view having AVG_PRICE_VIEW>=350000")

print("--- %s seconds ---" % (time.time() - start_time))
homeparq.show()


--- 0.057646751403808594 seconds ---
+--------------+----+
|AVG_PRICE_VIEW|view|
+--------------+----+
|    1056336.74|  85|
|     736679.93|  65|
|      755214.8|  53|
|    1080649.37|  78|
|    1053472.79|  81|
|    1058802.78|  76|
|    1137372.73|  91|
|    1026006.06|  93|
|     733780.26|  52|
|    1070444.25|  86|
|     1033536.2|  94|
|      734340.5|  57|
|     798684.82|  54|
|    1017815.92|  96|
|     970402.55|  92|
|     767036.67|  64|
|     746877.59|  61|
|    1031719.35|  88|
|     780914.67|  72|
|      791453.0|  59|
+--------------+----+
only showing top 20 rows



In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [32]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False